### Contributors: Brian Waweru
### Date        : 08th May, 2025

In [13]:
! ls
! pwd

Data-Preparation-for-Machine-Learning.txt
End-of-Phase-3-Specials-Notes.txt
Grading-End-of-Phase-3.txt
README.md
notebook.ipynb
phase-3-notebook-requirements.txt
presentation.pptx
/c/Users/rurig/OneDrive/GitFiles/phase-3/dsfpt10-p3-end-of-phase-3-project


In [21]:
# "Churn in Telecom's dataset".lower().replace(' ', '_')

import pandas as pd

# file = "/c/Users/rurig/OneDrive/GitFiles/phase-3/dsfpt10-p3-end-of-phase-3-project/churn_in_telecoms_dataset.csv"
file = "churn_in_telecoms_dataset.csv"
df = pd.read_csv(file)
# df.head()
df.shape


(3333, 21)

In [ ]:
df.